#Overview of Delivery Duration Prediction 

This project aims to show the expected time of delivery when a consumer places an order on a e-commerce site, as it has a big impact on consumer experience.

#Data Exploration 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp,unix_timestamp,col,sum,avg,countDistinct,mean
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [0]:
spark=SparkSession.builder.appName("DDP").getOrCreate()
file_loc="dbfs:/FileStore/tables/historical_data.csv"
df=spark.read.csv(file_loc, header=True, inferSchema=True)
df.show(3)

+---------+-------------------+--------------------+--------+----------------------+--------------+-----------+--------+------------------+--------------+--------------+---------------------+------------------+------------------------+------------------------------+--------------------------------------------+
|market_id|         created_at|actual_delivery_time|store_id|store_primary_category|order_protocol|total_items|subtotal|num_distinct_items|min_item_price|max_item_price|total_onshift_dashers|total_busy_dashers|total_outstanding_orders|estimated_order_place_duration|estimated_store_to_consumer_driving_duration|
+---------+-------------------+--------------------+--------+----------------------+--------------+-----------+--------+------------------+--------------+--------------+---------------------+------------------+------------------------+------------------------------+--------------------------------------------+
|        1|2015-02-06 22:24:17| 2015-02-06 23:27:16|    1845|   

In [0]:
df=df.withColumn("actual_delivery_time", to_timestamp("actual_delivery_time","yyyy-MM-dd HH:mm:ss"))
df=df.withColumn("actual_total_delivery_duration", unix_timestamp("actual_delivery_time")-unix_timestamp("created_at"))
df = df.withColumn("estimated_non_prep_duration",col("estimated_store_to_consumer_driving_duration")+col("estimated_order_place_duration"))
df = df.withColumn("busy_dashers_ratio",col("total_busy_dashers")/col("total_onshift_dashers"))
df.printSchema()
df.count()


root
 |-- market_id: string (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- actual_delivery_time: timestamp (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- store_primary_category: string (nullable = true)
 |-- order_protocol: string (nullable = true)
 |-- total_items: integer (nullable = true)
 |-- subtotal: integer (nullable = true)
 |-- num_distinct_items: integer (nullable = true)
 |-- min_item_price: integer (nullable = true)
 |-- max_item_price: integer (nullable = true)
 |-- total_onshift_dashers: string (nullable = true)
 |-- total_busy_dashers: string (nullable = true)
 |-- total_outstanding_orders: string (nullable = true)
 |-- estimated_order_place_duration: integer (nullable = true)
 |-- estimated_store_to_consumer_driving_duration: string (nullable = true)
 |-- actual_total_delivery_duration: long (nullable = true)
 |-- estimated_non_prep_duration: double (nullable = true)
 |-- busy_dashers_ratio: double (nullable = true)

Out[38]: 197428

In [0]:
tot_rows=df.count()
null_counts=df.select([sum(col(c).isNull().cast("int")).alias(c+"_nulls")for c in df.columns])
total_nulls=null_counts.selectExpr("+".join([c+"_nulls"for c in df.columns])).collect()[0][0]
missing_percent=(total_nulls/(tot_rows*len(df.columns)))*100
print("# Percentange of missing value % :", missing_percent)

avg_delivery_sec=df.select(avg("actual_total_delivery_duration")).collect()[0][0]
avg_delivery_min=avg_delivery_sec/60
print("# Average order delivery time in minutes:",avg_delivery_min,"Minutes")

# Percentange of missing value % : 0.5442890306179574
# Average order delivery time in minutes: 48.4709562305935 Minutes


In [0]:
df.select([countDistinct(col(c)).alias(c) for c in df.columns]).show()

In [0]:
mode_value=(df.groupBy("store_primary_category").count().orderBy("count",ascending=False).first()[0])
df = df.fillna({"store_primary_category":mode_value})
columns = [
    'total_onshift_dashers',
    'total_busy_dashers',
    'total_outstanding_orders',
    'estimated_store_to_consumer_driving_duration',
    'estimated_non_prep_duration'
]

# Step 1: Calculate mean for each column
means = df.select([mean(col(c)).alias(c) for c in columns]).collect()[0].asDict()

# Step 2: Fill nulls with the respective mean values
df_filled = df.fillna(means)
df_filled.describe().show()

+-------+------------------+-----------------+----------------------+------------------+------------------+------------------+------------------+-----------------+------------------+---------------------+------------------+------------------------+------------------------------+--------------------------------------------+------------------------------+---------------------------+-------------------+
|summary|         market_id|         store_id|store_primary_category|    order_protocol|       total_items|          subtotal|num_distinct_items|   min_item_price|    max_item_price|total_onshift_dashers|total_busy_dashers|total_outstanding_orders|estimated_order_place_duration|estimated_store_to_consumer_driving_duration|actual_total_delivery_duration|estimated_non_prep_duration| busy_dashers_ratio|
+-------+------------------+-----------------+----------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------